In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
G

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-30 05:52:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-30 05:52:17 (6.03 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.|2015-01-31 00:08:00|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...| 

In [ ]:
# Count the number of the reviews 
print(df.count())

3093869


In [ ]:
# Drop duplicates and incomplete rows 
dropped_df = df.dropna()
print(dropped_df.count())

clean_df = dropped_df.dropDuplicates()
print(clean_df.count())

3093660
3093660


In [ ]:
# Print schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:

# Changing datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

clean_df = clean_df.withColumn("customer_id", clean_df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", clean_df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", clean_df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", clean_df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", clean_df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", clean_df["total_votes"].cast(IntegerType()))\

# Print schema
clean_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create review dataframe to match review_id_table table
review_id_df = clean_df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100BOZN6L781S|   16879807|B001DMJ49C|     283132125|2011-01-15 00:02:00|
|R100FFUB2P1J93|   50051109|B0052SCU8U|     572574607|2012-01-30 00:12:00|
|R100GHF5YMF7F2|   53095663|B000EPNDEG|     472710979|2007-01-22 00:01:00|
|R100ILOHP2RE00|   52488398|B001FA1NUK|      30849558|2010-01-28 00:09:00|
|R100P56XBWHN6A|   43084076|B0080R95XI|         87429|2015-01-22 00:01:00|
|R102FREZLGEIYO|   23995515|B007B8S4JI|     361114123|2012-01-30 00:10:00|
|R102GKAR5L7TFF|   47949431|B0037FSWGY|     900543314|2013-01-26 00:04:00|
|R102K17WSFXOW7|   30299540|B0058O8GUO|     964606444|2013-01-01 00:08:00|
|R102XT9ZJW4SBU|   23399623|B0041MMMWW|     431770258|2015-01-20 00:01:00|
|R1033ERO22KHSJ|   32651504|9983891212|     822800427|2010-01-03 00:12:00|
|R103JTN0A0HBU8|   512453

In [ ]:
# Print schema
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create customer dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = clean_df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49266466|           234|
|   53075795|           200|
|   30669680|           190|
|   53037408|           180|
|   50820654|           171|
|   52938899|           166|
|   44834233|           159|
|   50027179|           159|
|   39789300|           154|
|   32038204|           149|
|   53090839|           140|
|   52340667|           135|
|   50530418|           132|
|   51346302|           132|
|   52870270|           118|
|   12200139|           117|
|   52643790|           113|
|   51812418|           112|
|   52460215|           112|
|   50529300|           107|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create product dataframe to match products table(only unique values)
products_df = clean_df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00192G76O|Cowon D2 - 16 GB ...|
|B00JDYMPYS|Sylvania SKCR2713...|
|B000OKH6ZI|Panasonic Noise-C...|
|B0013P3ZOE|Jvc(r) Harx900 Dy...|
|B008VMT2HQ|TDK Life on Recor...|
|B00125UBUG|Sennheiser Dynami...|
|B001EZUQ5E|Sennheiser Twist-...|
|B00004SCIX|GE 72946 TV Weath...|
|B003Z3G66G|Sony Digital Musi...|
|B004U7PSEM|DIRECTV RC65 4-De...|
|B0097BEE9Q|Apple iPod nano 1...|
|B00GX66YJG|Kanto  Mobile TV ...|
|B007SNM046|SimplyVibe V3-5CP...|
|B005LJQMOS|BlueRigger HDMI C...|
|B002I529AY|Kicker iK501 Digi...|
|B003Y7PXSK|iTour-POP Ultra P...|
|B001F08BSC|Memorex - 50 x DV...|
|B003XQLUDY|Sangean AM/FM-RDS...|
|B002BDTLWM|Creative Labs Zen...|
|B00009QOV7|Pioneer CTW208R D...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create vine dataframe to match vine_table table

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = clean_df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100BOZN6L781S|          5|            0|          0|   N|
|R100FFUB2P1J93|          4|            0|          0|   N|
|R100GHF5YMF7F2|          4|            6|          9|   N|
|R100ILOHP2RE00|          5|            0|          0|   N|
|R100P56XBWHN6A|          5|            0|          0|   N|
|R102FREZLGEIYO|          4|            0|          0|   N|
|R102GKAR5L7TFF|          4|            3|          3|   N|
|R102K17WSFXOW7|          1|            1|          1|   N|
|R102XT9ZJW4SBU|          1|            0|          0|   N|
|R1033ERO22KHSJ|          5|            0|          0|   N|
|R103JTN0A0HBU8|          3|            1|          1|   N|
|R103PGUT9IOR9H|          2|            0|          0|   N|
|R105285Q1MJBRT|          5|            2|          3|   N|
|R106DJAJRF8AUF|          5|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "X"
config = {"user":"X",
          "password": "X",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)